In [26]:
from lusidtools.jupyter_tools import toggle_code

"""Structured Results Store for storage of Portfolio data

Attributes
----------
structured_results_store
virtual_document
luminesce
"""

toggle_code("Toggle Docstring")

# Structured Results Store Example

This notebook demostrates loading of a custom Portfolio dataset into the LUSID Structured Results Store.

For more context on what a Structure Result Store/Data is see KB Article <a ref=https://support.lusid.com/knowledgebase/article/KA-01893/en-us>KA-01893</a>

Once loaded, the dataset can be retrieved as one document, or the individual fields accessed

In [27]:
import os
import pandas as pd
from datetime import datetime, timezone
import io
import json
from IPython.core.display import HTML
from itertools import chain

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package
from lusid.utilities import ApiClientFactory

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.9423.0,0.5.2891,"{'relation': 'RequestLogs', 'href': 'http://se..."


In [28]:
srs_api = api_factory.build(lu.StructuredResultDataApi)

<h2>Scope, Effective Date, Result Type</h2>

<ul>
<li>Scope: The scope in which to create or update data maps.</li>
<li>Effective Date: Date at which the structured result is effective from.</li>
<li>Result Type: The Class of the Result, has to be one of [UnitResult/Analytic, UnitResult/Grouped, UnitResult/Holding]</li>
</ul>

In [29]:
scope = "srs" 
effective_date = datetime(2021, 1, 27, tzinfo=timezone.utc)
sample_data_result_type = "UnitResult/Custom" 

## Define functions to create the Data Map


<h3>Data Mapping</h3>

The Structured Result Store allows us to upsert structured results, but to give them structure we must first define a Data Mapping,

You can think of a Data Mapping as defining the columns and the column types for each row that gets upserted into the SRS using a specific Data Map

Each "row" can have multiple keys and when difining the data map there are 4 different key types, Unique, PartOfUnique, Leaf, and CompositeLeaf

<h4>Unique</h4>
<blockquote>A primary key that will throw a unique key constraint if multiple equal values are upserted for a single specific result type</blockquote>
<h4>Part of Unique</h4>
<blockquote>A key that will be considered as part of the compoisite primary key for an entity</blockquote>
<h4>Leaf</h4>
<blockquote>Leaf define a value that wont be considered as part of the primary or primary composite key.</blockquote>
<h4>Composite Leaf</h4>
<blockquote>
CompositeLeaf is an abstraction that allows the user to specify which Leafs should be connected, they define data types rather than names of entities as they represent a group of entities.
E.g. If we have an Accural we can define a composite leaf as two leafs of amount and currency
<p></p>
<blockquote>
"UnitResult/Accrual"

DataDefinition(address="UnitResult/Accrual", dataType= "Result0D", keyType="CompositeLeaf") 
</blockquote>

<blockquote>
"UnitResult/Accrual/Amount"

DataDefinition(address="UnitResult/Accrual/Amount", name="Accrual", dataType= "decimal", keyType="Leaf")
</blockquote>
<blockquote>
"UnitResult/Accrual/Ccy"

DataDefinition(address="UnitResult/Accrual/AmountCcy", name="AccrualCcy", dataType= "string", keyType="Leaf")
</blockquote>
</blockquote>

In [30]:
def gen_srs_data_map(leaf_columns, unique_columns, data_types):
    def define_data_definitions(columns, key_type):
        return [lm.DataDefinition(address=f"UnitResult/{scope}/{column}", name=column, data_type=data_types[column], key_type=key_type) 
                      for column in columns]
    data_map = [define_data_definitions(columns, key_type) for columns, key_type in zip((leaf_columns, unique_columns), ("Leaf", "PartOfUnique"))]

    return lm.DataMapping(data_definitions = list(chain(*data_map)))

def create_data_map(df, code, version, leaf_columns, unique_columns, data_types):
    srs_data_map = gen_srs_data_map(leaf_columns, unique_columns, data_types)
    srs_data_map_key = lm.DataMapKey(version=version, code=code)
    
    try:    
        srs_api.create_data_map(
            scope=scope, 
            request_body={
                code: lm.CreateDataMapRequest(
                    id = srs_data_map_key,
                    data = srs_data_map
                )
            }
        )
    except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail['code'] not in [461]: 
            raise e
    return srs_data_map_key

## Read in sample data

In [31]:
df = pd.read_csv("data/srs_custom_data.csv")
df = df.round(0)
display(df.head(5))

,Port,Currency,Date,Sys,Weight,Amount,Class
0,PortA,AUD,20210713,FSC,10,"50,000,000.00",C
1,PortA,CAD,20210713,FSC,7,"138,234,286.00",C
2,PortA,CHF,20210713,FSC,3,"203,402.00",C
3,PortA,EUR,20210713,FSC,2,"235,402,950.00",C
4,PortA,EUR,20210715,FTC,3,"22,342,054.00",C


## Upsert data into the Structured Results Store

In [40]:
def insert_and_load_data(dataFrame):
    srs_ids=[]
    
    version = "1.01"
    portfolios = dataFrame.groupby("Port")
    unique_columns = ["Currency", "Date", "Sys", "Class"]
    leaf_columns = [column for column in dataFrame.columns if column not in unique_columns]
    
    sample_data_map_key = create_data_map(
        df = df,
        code = 'sample_data_map',
        version = "1.03",
        leaf_columns = leaf_columns,
        unique_columns = unique_columns,
        data_types = { "RowId": "string", 
                       "Port": "string",
                       "Currency": "string",
                       "Date": "string",
                       "Sys": "string",
                       "Weight": "decimal",
                       "Amount": "decimal",
                       "Class": "string"}
    )

    for portfolio_id, pf_df in portfolios:    
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
    
        srs_ids.append(srs_id) 
    
        csv_data = io.StringIO()
        pf_df.to_csv(csv_data)   
        
        request_body = {
            portfolio_id: lm.UpsertStructuredResultDataRequest(
                id = srs_id,
                data = lm.StructuredResultData(
                    document_format = "csv",
                    version = version,
                    name = "Data file",
                    document = csv_data.getvalue(),
                    data_map_key = sample_data_map_key
                )
            )
        }
        
        result = srs_api.upsert_structured_result_data(
            scope=scope,
            request_body=request_body)

        display(pd.DataFrame(result.values.items()))
        
    return srs_ids


srs_ids = insert_and_load_data(df)

,0,1
0,PortA,2022-06-13 13:39:14.289462+00:00


,0,1
0,PortB,2022-06-13 13:39:16.025411+00:00


,0,1
0,PortC,2022-06-13 13:39:17.485248+00:00


## Retrieve the raw data from the Structured Results Store

In [33]:
def retrieve_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
    
        csv_data = io.StringIO(result.values["key"].document)
        doc = pd.read_csv(csv_data)
        display(srs_id.code)
        display(doc)
        
retrieve_data()

'PortA'

,Unnamed: 0,Port,Currency,Date,Sys,Weight,Amount,Class
0,0,PortA,AUD,20210713,FSC,10,"50,000,000.00",C
1,1,PortA,CAD,20210713,FSC,7,"138,234,286.00",C
2,2,PortA,CHF,20210713,FSC,3,"203,402.00",C
3,3,PortA,EUR,20210713,FSC,2,"235,402,950.00",C
4,4,PortA,EUR,20210715,FTC,3,"22,342,054.00",C
5,5,PortA,EUR,20210727,FTC,5,"2,349,130,942.00",C
6,6,PortA,EUR,20210812,FTC,4,"23,489,234.00",C
7,7,PortA,GBP,20210713,FSC,6,"83,425,656.00",C
8,8,PortA,GBP,20210812,FTC,9,"54,354,564.00",C
9,9,PortA,JPY,20210713,FSC,9,"654,659.00",C


'PortB'

,Unnamed: 0,Port,Currency,Date,Sys,Weight,Amount,Class
0,26,PortB,AUD,20210713,FSC,10,"5,590,075.00",B
1,27,PortB,AUD,20210812,FTC,10,"5,453,435.00",B
2,28,PortB,SGD,20210713,FSC,8,"43,545,554.00",B
3,29,PortB,USD,20210713,FSC,9,"543,543,000,000.00",B
4,30,PortB,EUR,20210713,FSC,8,"54,545,435.00",B


'PortC'

,Unnamed: 0,Port,Currency,Date,Sys,Weight,Amount,Class
0,31,PortC,AUD,20210713,FSC,9,"75,174,369.00",C
1,32,PortC,AUD,20210715,FTC,1,"261,361.00",C
2,33,PortC,AUD,20210716,FTC,3,"73,055.00",C
3,34,PortC,AUD,20210719,FTC,2,"4,161,646.00",C
4,35,PortC,AUD,20210720,FTC,2,"787,878,787.00",C
5,36,PortC,AUD,20210722,FTC,5,"878,787,878.00",C
6,37,PortC,AUD,20210723,FTC,3,"3,349,533.00",C
7,38,PortC,AUD,20210728,FTC,7,"8,748,787.00",C
8,39,PortC,AUD,20210812,FTC,4,"4,864,848,616.00",C
9,40,PortC,CAD,20210713,FSC,10,"854,684,848.00",C


## Extract a 'Virtual Document' from the Structured Results Store

In [34]:
def retrieve_virtual_document():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_virtual_document(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
        
        # Convert to DataFrame
        result_dfs = []

        for item in result.values["key"].data:
            columns = item.row_data.columns
            values = [i.value for i in item.row_data.values]
    
            row_df = pd.DataFrame(values).T
            row_df.columns = columns
            
            for row in item.row_id.items():            
                row_df[row[0]] = [row[1]]

            result_dfs.append(row_df)
    
        all_dfs = pd.concat(result_dfs)
        
        return all_dfs
        
all_dfs = retrieve_virtual_document()

all_dfs.head(5)

,UnitResult/srs/Port,UnitResult/srs/Weight,UnitResult/srs/Amount,UnitResult/srs/Class,UnitResult/srs/Currency,UnitResult/srs/Date,UnitResult/srs/Sys
0,PortA,6.00,"16,875.00",C,USD,20210714,FTC
0,PortA,2.00,"235,402,950.00",C,EUR,20210713,FSC
0,PortA,4.00,"101,001.00",C,USD,20210723,FTC
0,PortA,10.00,"4,534.00",C,USD,20210726,FTC
0,PortA,9.00,"150,000.00",C,JPY,20210812,FTC


<h2>Using Luminecese to access data in the Structured Result Store</h2>

To access data in the SRS via luminecese we use the Lusid.UnitResult.AtomisedResult provider which takes a scope, code, source, resultType, and effectiveAt and asAt parameters are mandatory values.

<ul>
<li>Scope will determine where to look for the document e.g UnitResult/scope/...</li>
<li>Code + Source will uniquely define a document inside the specified scope</li>
<li>ResultType indicates the type of document</li>
<li>EffectiveAt and asAt are used to give time context to reterieving the document.</li>
</ul>

Its important to note that the SRS does not store information in a tabular form, and thus in order to massage it into tabular data we need to pass the result from the AtomisedResult provider through a pivot, to acheive this we use the Tools.Pivot provider.

In [50]:
import lumipy as lmpy

secrets_path = os.getenv("FBN_SECRETS_PATH")

client = lmpy.get_client(api_secrets_filename=secrets_path)

srs_client_filetered = client.query_and_fetch("""
@@effAt = select datetime('now');
@portfolio_data = use Lusid.UnitResult.AtomisedResult with @@effAt
  --scope=srs
  --code=PortA
  --source=Client
  --resultType=UnitResult/Custom
  --effectiveAt={@@effAt:s}
  --asAt={@@effAt:s}
  enduse;

@input = SELECT * FROM @portfolio_data WHERE `UnitResult/srs/Currency`=='USD';


@x =
select 
    DataKey,
    [UnitResult/srs/Class],
    [UnitResult/srs/Currency],
    [UnitResult/srs/Date],
    [UnitResult/srs/Sys],
    max(coalesce(ValueReal, ValueText)) as Value
from
    @input
group by
    1, 2, 3, 4, 5
    ;

@pivoted = 
use Tools.Pivot with @x
--aggregateColumns=Value
enduse;

select * from @pivoted;""")

srs_client_filetered.head(5)



Request to query and fetch failed with status code 400, reason: 'Bad Request)'.
    Details:
        Failed to parse: 
        
        pragma QueryName = 'query';
        -- lumipy 
        
        @@effAt = select datetime('now');
        @portfolio_data = use Lusid.UnitResult.AtomisedResult with @@effAt
          --scope=srs
          --code=PortA
          --source=Client
          --resultType=UnitResult/Custom
          --effectiveAt={@@effAt:s}
          --asAt={@@effAt:s}
          enduse;
        
        @input = 
        SELECT * FROM @portfolio_data WHERE `UnitResult/srs/Currency`▷︎==◁︎❴';',SCOL,'=',ASSIGN,'*',STAR,'+',PLUS,'-',MINUS,'||',PIPE2,'/',DIV,'%',MOD,'<<',LT2,'>>',GT2,'&',AMP,'|',PIPE,'<',LT,'<=',LT_EQ,'>',GT,'>=',GT_EQ,'==',EQ2,'!=',NOT_EQ1,'<>',NOT_EQ2,AND,BETWEEN,NOTBETWEEN,COLLATENOACCENT,COLLATENOCASE,COLLATEEXACT,EXCEPT,GLOB,NOTGLOB,GROUP,IN,NOTIN,INTERSECT,IS,ISNOT,ISNULL,ISNOTNULL,LIKE,NOTLIKE,LIMIT,OR,ORDER,REGEXP,NOTREGEXP,UNION,WINDOW❵'USD'
        
  

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 13 Jun 2022 15:24:01 GMT', 'Content-Type': 'application/problem+json; charset=utf-8', 'Content-Length': '2663', 'Connection': 'keep-alive', 'honeycomb-meta-success': 'False', 'honeycomb-meta-requestId': '0HMIBG8LC830B:00000015', 'lusid-meta-correlationId': '0HMIBG8LC830B:00000015', 'honeycomb-meta-duration': '297', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src data: 'self' https://fonts.googleapis.com https://fonts.gstatic.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src https://fonts.googleapis.com 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; connect-src https://finbourne.okta.com 'self' https://*.lusid.com https://*.f

## Cleanup - delete the data from the Structured Results Store

In [36]:
def delete_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.delete_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )

delete_data()